# DEEP CONVOLUTIONAL GENERATIVE ADVERSARIAL NETWORKS(DCGAN)
In the paper 'UNSUPERVISED REPRESENTATION LEARNING WITH DEEP CONVOLUTIONAL GENERATIVE ADVERSARIAL NETWORKS' by Alec Radford, Luke Metz and Soumith Chintala several ideas were introduced for unsupervised learning by CNN. The DCGAN like GAN has both the Generator and Discriminator.A generator is a network that tries to create fake data identical to real data and discriminator tries to distinguish between fake and real data. To give an example let's assume Genrator creates counterfeit money and it's the job of the Discriminator to distinguish between real money and counterfeit money. Over time the Generator becomes better in creating counterfeit money which is identical to real money.

The architecture guidelines from the DCGAN paper are:-
- Replace any pooling layers with strided convolutions (discriminator) and fractional-strided convolutions (generator).
- Use batchnorm in both the generator and the discriminator.
- Remove fully connected hidden layers for deeper architectures.
- Use ReLU activation in generator for all layers except for the output, which uses Tanh.
- Use LeakyReLU activation in the discriminator for all layers.

<img src='images/dcgan2.png'>
                                               <center><h2>DCGAN ARCHITECTURE</h2></center>


In [1]:
import tensorflow as tf   #importing Modules
import numpy as np
import os,imageio,itertools,time,pickle
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt

In [2]:
mnist=input_data.read_data_sets("MNIST_data/", one_hot=True,reshape=[]) #one hot true does one hot encoding
tf.reset_default_graph()
batch_size=50
lr=0.0002
epoch=100
display_epoch = 1
logs_path = 'dcgan'

## Batch Normalization

In [3]:
def batch_norm(x,phase_train=True):
    with tf.name_scope('bn'):
        normed=tf.cond(phase_train,lambda:tf.contrib.layers.batch_norm(x,epsilon=1e-5),lambda:tf.contrib.layers.batch_norm(x,epsilon=1e-5,is_training=False))
        return normed

## Discriminator
It takes input size of 64x64 image and after several strided convolution layers tries to predict whether the image is fake or real.

In [4]:
def discriminator(x,phase_train,reuse=False):
    with tf.variable_scope('discriminator', reuse=reuse):
        l1=tf.layers.conv2d(x, 128, [4, 4], strides=(2, 2), padding='same')
        l1=tf.nn.leaky_relu(l1)
        l2=tf.layers.conv2d(l1, 256, [4, 4], strides=(2, 2), padding='same')
        l2=batch_norm(l2,phase_train)
        l2=tf.nn.leaky_relu(l2)
        l3=tf.layers.conv2d(l2, 512, [4, 4], strides=(2, 2), padding='same')
        l3=batch_norm(l3,phase_train)
        l3=tf.nn.leaky_relu(l3)
        l4=tf.layers.conv2d(l3, 1024, [4, 4], strides=(2, 2), padding='same')
        l4=batch_norm(l4,phase_train)
        l4=tf.nn.leaky_relu(l4)
        l5=tf.layers.conv2d(l4, 1, [4, 4], strides=(1, 1), padding='valid')
        o=tf.nn.sigmoid(l5)
        return o,l5

## Generator
It starts from a completely random distribution and learns to fool the discriminator after going through several deconvolution operations along with upsampling. At the end it outputs a 64x64 image.

In [5]:
def generator(z_dim,phase_train,reuse=False):
    with tf.variable_scope('generator', reuse=reuse):
        l0=tf.layers.conv2d_transpose(z_dim, 1024, [4, 4], strides=(1, 1), padding='valid')
        l0=batch_norm(l0,phase_train)
        l0=tf.nn.relu(l0)
        l1=tf.layers.conv2d_transpose(l0, 512, [4, 4], strides=(2, 2), padding='same')
        l1=batch_norm(l1,phase_train)
        l1=tf.nn.relu(l1)
        l2=tf.layers.conv2d_transpose(l1, 256, [4, 4], strides=(2, 2), padding='same')
        l2=batch_norm(l2,phase_train)
        l2=tf.nn.relu(l2)
        l3=tf.layers.conv2d_transpose(l2, 128, [4, 4], strides=(2, 2), padding='same')
        l3=batch_norm(l3,phase_train)
        l3=tf.nn.relu(l3)
        l4=tf.layers.conv2d_transpose(l3, 1, [4, 4], strides=(2, 2), padding='same')
        l5=tf.nn.tanh(l4)
        return l5

## Visualization
The following few lines of code help in visualization.

In [8]:
root = 'MNIST_DCGAN_results/'
model = 'MNIST_DCGAN_'
if not os.path.isdir(root):
    os.mkdir(root)
if not os.path.isdir(root + 'Fixed_results'):
    os.mkdir(root + 'Fixed_results')

fixed_z_ = np.random.normal(0, 1, (batch_size, 1, 1, 100))
def show_result(num_epoch, show = False, save = False, path = 'result.png'):
    test_images = sess.run(g_i, {z_dim: fixed_z_, phase_train: False})

    size_figure_grid = 5
    fig, ax = plt.subplots(size_figure_grid, size_figure_grid, figsize=(5, 5))
    for i, j in itertools.product(range(size_figure_grid), range(size_figure_grid)):
        ax[i, j].get_xaxis().set_visible(False)
        ax[i, j].get_yaxis().set_visible(False)

    for k in range(size_figure_grid*size_figure_grid):
        i = k // size_figure_grid
        j = k % size_figure_grid
        ax[i, j].cla()
        ax[i, j].imshow(np.reshape(test_images[k], (64, 64)), cmap='gray')

    label = 'Epoch {0}'.format(num_epoch)
    fig.text(0.5, 0.04, label, ha='center')

    if save:
        plt.savefig(path)

    if show:
        plt.show()
    else:
        plt.close()

In [6]:
with tf.name_scope('inputs') as inputs: #Inputs of the Network
        x = tf.placeholder(tf.float32, shape=(None,64,64,1), name='x')
        z_dim = tf.placeholder(tf.float32, shape=(None,1,1,100), name='z_dim')
        phase_train = tf.placeholder(tf.bool, name='phase_train')

In [7]:
with tf.name_scope('Generator'): 
    g = generator(z_dim,phase_train,reuse=False)# Takes a random distribution(z_dim) as input and returns a 64x64 image.

with tf.name_scope('Generator_I'): #Visualization Operation, Uses the previous trained variables (reuse=True).   
    g_i= generator(z_dim,phase_train,reuse=True)
    
gs=tf.summary.image('Generator',g, max_outputs=10) #Tensoroard Visualization Op

with tf.name_scope('Discriminator_T'): #Real Data as Input
    d_t ,d_tlog= discriminator(x,phase_train,reuse=False)
with tf.name_scope('Discriminator_F'): #Fake Data as Input
    d_f ,d_flog= discriminator(g,phase_train,reuse=True)
#Tensorboard_Op
d_fs=tf.summary.histogram("Discriminator_FACT",d_f)
d_ts=tf.summary.histogram("Discriminator_TACT",d_t)

In [9]:
with tf.name_scope('Discriminator_Loss_Fake'): #The Discriminator will set Generator's Fake Data as 0
    dfcost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_flog, labels=tf.zeros_like(d_f)))
    dfcosts=tf.summary.scalar("Discriminator_Loss_Fake",dfcost)

with tf.name_scope('Discriminator_Loss_Real'): #The Discriminator will set Real Data input as 1, 0.9 is multiplied for label smoothning.
    dtcost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_tlog, labels= tf.ones_like(d_t)*0.9))
    dtcosts=tf.summary.scalar("Discriminator_Loss_Real",dtcost)

with tf.name_scope('Discriminator_Loss'):#The Two losses are added
    dcost=dtcost+dfcost
    dcosts=tf.summary.scalar('Discriminator_Loss',dcost)

In [10]:
with tf.name_scope('Generator_Loss'): #The Generator will try to fool the Discriminator 
    gcost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_flog, labels=tf.ones_like(d_f)))
    gcosts=tf.summary.scalar('Generator_Loss',gcost)

In [13]:
T_vars = tf.trainable_variables() #Making list of Trainable Variables
D_vars = [var for var in T_vars if var.name.startswith('discriminator')]
G_vars = [var for var in T_vars if var.name.startswith('generator')]

with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): #Using Adam Optimizer
    D_optim = tf.train.AdamOptimizer(lr, beta1=0.5).minimize(dcost, var_list=D_vars)
    G_optim = tf.train.AdamOptimizer(lr, beta1=0.5).minimize(gcost, var_list=G_vars)
#Tensorboard Operation
d_summary=tf.summary.merge([dcosts,])
g_summary=tf.summary.merge([gcosts,gs])

In [14]:
run_config = tf.ConfigProto()
run_config.gpu_options.allow_growth=True
with tf.Session(config=run_config) as sess:
    sess.run(tf.global_variables_initializer())
    summary_writer = tf.summary.FileWriter(logs_path)  #WritesTheGraphForTensorboard
    summary_writer.add_graph(graph=sess.graph)        
    gstep=1
    train_set=tf.image.resize_images(mnist.train.images, [64, 64]).eval(session=sess)
    train_set=(train_set-0.5)/0.5
    for i in range(epoch):
            g_loss = []
            d_loss = []            #calculate loss
            for j in range(int(mnist.train.num_examples/batch_size)):
                epoch_x = train_set[j*batch_size:(j+1)*batch_size]
                #epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                #epoch_x=(epoch_x-0.5)/0.5
                #epoch_x=np.resize(np.reshape(epoch_x,[-1,28,28,1]),[batch_size,64,64,1])
                z=np.random.normal(0, 1, (batch_size, 1, 1, 100))
                _,d_l,summary1=sess.run([D_optim,dcost,d_summary],feed_dict={x:epoch_x,z_dim:z,phase_train:True})
                d_loss.append([d_l])
                summary_writer.add_summary(summary1,gstep)
                z=np.random.normal(0, 1, (batch_size, 1, 1, 100))
                _,g_l,summary2 = sess.run([G_optim,gcost,g_summary],feed_dict={x:epoch_x,z_dim:z,phase_train:True})
                g_loss.append([g_l])
                summary_writer.add_summary(summary2,gstep) #Writes Summary
                gstep+=1
            print ("epoch done")
            fixed_p = root + 'Fixed_results/' + model + str(i + 1) + '.png'
            show_result((i + 1), save=True, path=fixed_p)
            print('Epoch', i, 'completed out of',epoch,'generator loss:',np.mean(g_loss),'discriminator loss:',np.mean(d_loss))

images = [] #Gif Creation
for e in range(epoch):
    img_name = root + 'Fixed_results/' + model + str(e + 1) + '.png'
    images.append(imageio.imread(img_name))
imageio.mimsave(root + model + 'generation_animation.gif', images, fps=5)


## The Generated Samples
<img src='images\MNIST_DCGAN_1.png'> <img src='images\MNIST_DCGAN_100.png'> <img src='images\MNIST_DCGAN2generation_animation.gif'>

In [15]:
print("Run the command line:\n" \
          "--> tensorboard --logdir=dcgan " \
          "\nThen open http://0.0.0.0:6006/ into your web browser")

Run the command line:
--> tensorboard --logdir=dcgan 
Then open http://0.0.0.0:6006/ into your web browser


## The Graph Of The Network
<img src='images\dcgan.png'>

## The Discriminator And Generator Losses
<img src='images\Capture.PNG'>